In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DataFrame 병합하기

## concat: 동일한 컬럼명 가지는 데이터프레임끼리 합치기
```
pandas.concat([A, B]) #A, B 데이터프레임 합치기(row 합치기)
pandas.concat([A, B], axis=1) #A, B 데이터프레임 합치기(컬럼 합치기 = 컬럼 추가)
```



In [2]:
import pandas as pd
import numpy as np

### 1) column 명이 같은 2개 데이터프레임 합치기
- ignore_index: 병합 시 기존 index 무시 여부 설명
    - False: 기존 인덱스 유지 / True: 기존 인덱스 무시(인덱스 재배열)
- axis:
    - 0: 위아래로 합치기
    - 1: column 합치기

In [9]:
#데이터프레임 생성: 컬럼명 같지만 인덱스 번호는 다르다
df1 = pd.DataFrame({'key1':[0,1,2,3,4], 'value1':['a','b','c','d','e']}, index=[0,1,2,3,4])
df2 = pd.DataFrame({'key1':[5,6,7,8,9], 'value1':['f','g','h','i','j']}, index=[3,4,5,6,7])

In [10]:
df1

,key1,value1
0,0,a
1,1,b
2,2,c
3,3,d
4,4,e


In [11]:
df2

,key1,value1
3,5,f
4,6,g
5,7,h
6,8,i
7,9,j


In [13]:
pd.concat([df1, df2]) #위아래로 합치기, 인덱스 원본 유지

,key1,value1
0,0,a
1,1,b
2,2,c
3,3,d
4,4,e
3,5,f
4,6,g
5,7,h
6,8,i
7,9,j


In [14]:
pd.concat([df1, df2], ignore_index=True, axis=1) #좌우로 합치기, 인덱스 재배열

,0,1,2,3
0,0.0,a,NaN,NaN
1,1.0,b,NaN,NaN
2,2.0,c,NaN,NaN
3,3.0,d,5.0,f
4,4.0,e,6.0,g
5,NaN,NaN,7.0,h
6,NaN,NaN,8.0,i
7,NaN,NaN,9.0,j


### 2) column명이 다른 경우

In [15]:
df3 = pd.DataFrame({'key2':[0,1,2,3,4], 'value2':['a','b','c','d','e'], 'a':[10,20,30,40,50]}, index=[0,1,2,3,4])
df4 = pd.DataFrame({'key3':[5,6,7,8,9], 'value3':['f','g','h','i','j'], 'a':[80,90,100,110,120]}, index=[3,4,5,6,7])

In [16]:
df3

,key2,value2,a
0,0,a,10
1,1,b,20
2,2,c,30
3,3,d,40
4,4,e,50


In [17]:
df4

,key3,value3,a
3,5,f,80
4,6,g,90
5,7,h,100
6,8,i,110
7,9,j,120


In [18]:
pd.concat([df3, df4]) #위아래로 합치기, 인덱스 원본 유지

###### 'a' 컬럼만 중복되므로 데이터가 완전함 ######

,key2,value2,a,key3,value3
0,0.0,a,10,NaN,NaN
1,1.0,b,20,NaN,NaN
2,2.0,c,30,NaN,NaN
3,3.0,d,40,NaN,NaN
4,4.0,e,50,NaN,NaN
3,NaN,NaN,80,5.0,f
4,NaN,NaN,90,6.0,g
5,NaN,NaN,100,7.0,h
6,NaN,NaN,110,8.0,i
7,NaN,NaN,120,9.0,j


In [19]:
pd.concat([df3, df4], axis=1) #컬럼 합치기, 인덱스 같은게 3과 4니까 여기만 데이터 완전함

,key2,value2,a,key3,value3,a
0,0.0,a,10.0,NaN,NaN,NaN
1,1.0,b,20.0,NaN,NaN,NaN
2,2.0,c,30.0,NaN,NaN,NaN
3,3.0,d,40.0,5.0,f,80.0
4,4.0,e,50.0,6.0,g,90.0
5,NaN,NaN,NaN,7.0,h,100.0
6,NaN,NaN,NaN,8.0,i,110.0
7,NaN,NaN,NaN,9.0,j,120.0


## merge: 두 데이터프레임을 공통된 컬럼 기준으로 합치기

- How 옵션
    - inner: 교집합
    - left: 왼쪽 기준으로 합치기(=왼쪽 데이터는 무조건 모두 출력, 오른쪽은 NaN 가능)
    - right: 오른쪽 기준으로 합치기(=left 반대)
    - outer: 합집합

```
pandas.merge(A, B, how='inner', on='Item_id') #A, B 데이터프레임에서 Item_id 기준으로 합치기
```



### 1) merge 기본

In [21]:
# customer, orders 데이터 프레임 생성
# '고객ID' 컬럼만 공통으로 존재

customer = pd.DataFrame({'고객ID':np.arange(6),
                         '이름':['철수','영희','길동','영수','수민','동건'],
                         '나이':[40,20,21,30,31,18]})

orders = pd.DataFrame({'고객ID':[1,1,2,2,2,3,3,1,4,9],
                       '상품명':['치약','칫솔','이어폰','헤드셋','수건','생수','수건','치약','생수','케이스'],
                       '수량':[1,2,1,1,3,2,2,3,2,1]})

In [22]:
customer

,고객ID,이름,나이
0,0,철수,40
1,1,영희,20
2,2,길동,21
3,3,영수,30
4,4,수민,31
5,5,동건,18


In [23]:
orders

,고객ID,상품명,수량
0,1,치약,1
1,1,칫솔,2
2,2,이어폰,1
3,2,헤드셋,1
4,2,수건,3
5,3,생수,2
6,3,수건,2
7,1,치약,3
8,4,생수,2
9,9,케이스,1


In [27]:
pd.merge(customer, orders, on='고객ID') #기본 merge: how='inner' / 즉 아래 코드와 동일함
pd.merge(customer, orders, how='inner', on='고객ID')

,고객ID,이름,나이,상품명,수량
0,1,영희,20,치약,1
1,1,영희,20,칫솔,2
2,1,영희,20,치약,3
3,2,길동,21,이어폰,1
4,2,길동,21,헤드셋,1
5,2,길동,21,수건,3
6,3,영수,30,생수,2
7,3,영수,30,수건,2
8,4,수민,31,생수,2


In [28]:
pd.merge(customer, orders, how='left', on='고객ID') #왼쪽에 해당하는 customer 모두 출력, orders는 일부 NaN

##### 고객ID '9'는 왼쪽엔 없고 오른쪽에만 있어서 무시됨 #####

,고객ID,이름,나이,상품명,수량
0,0,철수,40,NaN,NaN
1,1,영희,20,치약,1.0
2,1,영희,20,칫솔,2.0
3,1,영희,20,치약,3.0
4,2,길동,21,이어폰,1.0
5,2,길동,21,헤드셋,1.0
6,2,길동,21,수건,3.0
7,3,영수,30,생수,2.0
8,3,영수,30,수건,2.0
9,4,수민,31,생수,2.0


In [29]:
pd.merge(customer, orders, how='right', on='고객ID')

##### 고객ID '5'는 왼쪽엔 있고 오른쪽에는 없어서 무시됨 #####

,고객ID,이름,나이,상품명,수량
0,1,영희,20.0,치약,1
1,1,영희,20.0,칫솔,2
2,2,길동,21.0,이어폰,1
3,2,길동,21.0,헤드셋,1
4,2,길동,21.0,수건,3
5,3,영수,30.0,생수,2
6,3,영수,30.0,수건,2
7,1,영희,20.0,치약,3
8,4,수민,31.0,생수,2
9,9,NaN,NaN,케이스,1


In [30]:
pd.merge(customer, orders, how='outer', on='고객ID')

,고객ID,이름,나이,상품명,수량
0,0,철수,40.0,NaN,NaN
1,1,영희,20.0,치약,1.0
2,1,영희,20.0,칫솔,2.0
3,1,영희,20.0,치약,3.0
4,2,길동,21.0,이어폰,1.0
5,2,길동,21.0,헤드셋,1.0
6,2,길동,21.0,수건,3.0
7,3,영수,30.0,생수,2.0
8,3,영수,30.0,수건,2.0
9,4,수민,31.0,생수,2.0


### 질문) 가장 많이 팔린 상품명은 무엇?
- merge, groupby, sort_values 활용

In [41]:
solution_df = pd.merge(customer, orders, how='right', on='고객ID') #why right? 많이 팔린 '상품명'이 있어야 하므로, 오른쪽 df는 무조건 있어야 하니깐
solution_df

,고객ID,이름,나이,상품명,수량
0,1,영희,20.0,치약,1
1,1,영희,20.0,칫솔,2
2,2,길동,21.0,이어폰,1
3,2,길동,21.0,헤드셋,1
4,2,길동,21.0,수건,3
5,3,영수,30.0,생수,2
6,3,영수,30.0,수건,2
7,1,영희,20.0,치약,3
8,4,수민,31.0,생수,2
9,9,NaN,NaN,케이스,1


In [42]:
solution_df.groupby('상품명').sum()['수량'].sort_values(ascending=False)

,수량
상품명,
수건,5
생수,4
치약,4
칫솔,2
이어폰,1
케이스,1
헤드셋,1


### 질문) 영희가 가장 많이 구매한 상품은? / 피봇테이블까지 해보기

In [47]:
solution_df2 = pd.merge(customer, orders, how='inner', on='고객ID') #'고객'이 구매한 '상품명'이 있어야 하므로 inner
solution_df2

,고객ID,이름,나이,상품명,수량
0,1,영희,20,치약,1
1,1,영희,20,칫솔,2
2,1,영희,20,치약,3
3,2,길동,21,이어폰,1
4,2,길동,21,헤드셋,1
5,2,길동,21,수건,3
6,3,영수,30,생수,2
7,3,영수,30,수건,2
8,4,수민,31,생수,2


In [49]:
solution_df2.groupby(['이름', '상품명']).sum()['수량'].loc['영희']

,수량
상품명,
치약,4
칫솔,2


In [53]:
solution_df2

,고객ID,이름,나이,상품명,수량
0,1,영희,20,치약,1
1,1,영희,20,칫솔,2
2,1,영희,20,치약,3
3,2,길동,21,이어폰,1
4,2,길동,21,헤드셋,1
5,2,길동,21,수건,3
6,3,영수,30,생수,2
7,3,영수,30,수건,2
8,4,수민,31,생수,2


In [54]:
#pivot_table까지~~
solution_df2.pivot_table(index='이름', columns='상품명', values='수량', aggfunc='sum')

상품명,생수,수건,이어폰,치약,칫솔,헤드셋
이름,,,,,,
길동,NaN,3.0,1.0,NaN,NaN,1.0
수민,2.0,NaN,NaN,NaN,NaN,NaN
영수,2.0,2.0,NaN,NaN,NaN,NaN
영희,NaN,NaN,NaN,4.0,2.0,NaN
